In [3]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from keras.models import Sequential
from keras.layers import Dense

import re

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string

import numpy as np
import pandas as pd
!pip install --upgrade category_encoders
import category_encoders as ce 
from sklearn.model_selection import train_test_split
import gensim
from gensim import corpora
from keras.layers import Dense, Embedding, LSTM
import re

import category_encoders as ce

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import KFold, cross_val_predict, cross_val_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

import gensim
from gensim import corpora

Using TensorFlow backend.


     |████████████████████████████████| 92kB 28.2MB/s 


In [4]:
"""
define a function for auto ml
define a function to prepare data
define a function to clean text (also define stopwords, lemmatizer, punctuations)
"""
import nltk
nltk.download('stopwords')
def auto_ML(model, features, Xtrain, Ytrain, Xtest, Ytest, folds= 10):
    model.fit(Xtrain[features], Ytrain)
    prediction= model.predict(Xtest[features])
    
    # Cross validation
    scores= cross_val_score(model, Xtrain[features], Ytrain, cv= folds)
    val_score= sum(scores/folds)
    
    print('Test prediction:\n{}'.format(prediction))
    print('-------------------------------------')
    print('Accuracy score: {}'.format(accuracy_score(Ytest, prediction)))
    print('-------------------------------------')
    print('confusion matrix:\n {}'.format(confusion_matrix(Ytest, prediction)))
    print('-------------------------------------')
    print('Cross Validation score: {}'.format(val_score))
    
    correct_df= pd.DataFrame(columns= Xtrain.columns)
    incorrect_df= pd.DataFrame(columns= Xtrain.columns)
    
    for i, pred in enumerate(prediction):
        if pred== Ytest[i]:
            correct_df= correct_df.append(Xtest.loc[i, Xtest.columns])
        else:
            incorrect_df= incorrect_df.append(Xtest.loc[i, Xtest.columns])
            
    return correct_df, incorrect_df
    
    
def prepare_data(data, split_ratio):
    split= int(data.shape[0]*split_ratio)
    
    X_train= data.loc[:split, :]
    X_test= data.loc[split: , :]
    
    print('train shape: {}'.format(X_train.shape))
    print('test shape: {}'.format(X_test.shape))
    
    return X_train, X_test



stop= stopwords.words('english')
exclude = string.punctuation 
lemma = WordNetLemmatizer()

def clean(doc):
    stop_free= ' '.join(word for word in doc.lower().split() if word not in stop)
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized
    

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
train =pd.read_csv('gdrive/My Drive/train.csv')
test = pd.read_csv('gdrive/My Drive/test.csv')

In [7]:
print('training data:')
train.shape

training data:


(8160, 7)

In [8]:
print('testing data: ')
test.shape

testing data: 


(306, 6)

### add new features

In [9]:
"""
Add new feature: Question length

Add new features: Extract max IR score and correscponding page from IR_Wiki Scores
Also calculate difference between highest and second highest IR score

Add target feature paren_match (true if wiki page and answer match)

We will add the following features after splitting into training and testing.
1. Apply Weight of Evidence(WOE) encoding to category variable
2. Topic modeling
"""

#Question length (training data)
lens= []
for i in range(0, train.shape[0]):
    lens.append(len(train.loc[i]['Question Text']))
lens
train['Quest len']= lens
print('Question length created for training data')

#Question length (testing data)
lens= []
for i in range(0, test.shape[0]):
    lens.append(len(test.loc[i]['Question Text']))
lens
test['Quest len']= lens
print('Question length created for testing data')



######################################################################

# Max IR score and corresponding page (training data)
wiki_page= []
wiki_page2= []
page_score= []
diff= []
for i in range(0, train.shape[0]):
    ans_score= {}
    for ii in train.loc[i]['IR_Wiki Scores'].split(', '):
        ans_score[ii.split(':')[0]]= float(ii.split(':')[1])
        
    
    page= sorted(ans_score, key= ans_score.get, reverse= True)[0]
    page2= sorted(ans_score, key= ans_score.get, reverse= True)[1]
    wiki_page.append(page)
    page_score.append(ans_score[page])
    wiki_page2.append(page2)
    diff.append(ans_score[page]- ans_score[page2])

train['Wiki_page']= wiki_page
train['Wiki_page_embed']= wiki_page
train['Wiki_page2']= wiki_page2
train['Page score']= page_score
train['Score difference']= diff
print('Max IR score and corresponding page created for training data')



# Max IR score and corresponding page (testing data)
wiki_page= []
wiki_page2= []
page_score= []
diff= []
for i in range(0, test.shape[0]):
    ans_score= {}
    for ii in test.loc[i]['IR_Wiki Scores'].split(', '):
        ans_score[ii.split(':')[0]]= float(ii.split(':')[1])
        
    
    page= sorted(ans_score, key= ans_score.get, reverse= True)[0]
    page2= sorted(ans_score, key= ans_score.get, reverse= True)[1]
    wiki_page.append(page)
    wiki_page2.append(page2)
    page_score.append(ans_score[page])
    diff.append(ans_score[page]- ans_score[page2])

test['Wiki_page']= wiki_page
test['Wiki_page_embed']= wiki_page
test['Wiki_page2']= wiki_page2
test['Page score']= page_score
test['Score difference']= diff
print('Max IR score and corresponding page created for testing data')



################################################################

# Target feature paren_match (training data). It is 1 if answer and wiki page match. 0 otherwise
train['paren_match']= 0

for i, row in train.iterrows():
    if row['Answer'] == row['Wiki_page']:
        train.loc[i, 'paren_match']= 1
print('paren_match created for training data')



#################################################################

# Create a new column to embed categories (training data)
train['category_embed']= train['category']

"""
encoding= ce.WOEEncoder(cols= ['category_embed'], impute_missing= True)
encoding.fit(train[['category_embed']], train['paren_match'])
train['category_embed']=encoding.transform(train[['category_embed']])
print('Category embeddings created for training data')



# Create a new column to embed categories (testing data)
test['category_embed']= test['category']
test['category_embed']= encoding.transform(test[['category_embed']])
print('Category embeddings created for testing data')
"""



#################################################################


# Create new topic columns for topic modeling (training data)
train['Topic 0']= 0
train['Topic 1']= 0
train['Topic 2']= 0
train['Topic 3']= 0
print('Topics created for training data')


# Create new topic columns for topic modeling (testing data)
test['Topic 0']= 0
test['Topic 1']= 0
test['Topic 2']= 0
test['Topic 3']= 0
print('Topics created for testing data')


##################################################################
##################################################################

"""
Add new feature: topics (topic modeling)
We have 4 categories in the dataset. SO lets use 4 topics
"""

"""
# Preprocessing the text first
clean_questions= [clean(row['Question Text']).split() for _, row in train.iterrows()]
print('Text preprocessing done')

# Create a word to index mapping
dictionary= corpora.Dictionary(clean_questions)
print('Word index dictionary created')

# Convert the list of documents to a BOW matrix using the dictionary prepared above
doc_term_matrix = [dictionary.doc2bow(doc) for doc in clean_questions]
print('BOW matrix created')

# Train the LDA model on the document word matrix
ldamodel = gensim.models.ldamodel.LdaModel(doc_term_matrix, num_topics=4, id2word = dictionary, passes=50)
print('LDA model trained')
"""


"""
Add topic distributions as features to train and test data
"""

"""
# Topic distributions (training data)
for i, row in train.iterrows():
    for pair in ldamodel[doc_term_matrix[i]]:
        col= 'Topic '+ str(pair[0])
        train.loc[i, col]= pair[1]
        
print('Topic distributions added to training data')    
    

# Topic distributions (testing data)

clean_test= [clean(row['Question Text']).split() for _, row in test.iterrows()]
test_bow= [dictionary.doc2bow(doc) for doc in clean_test]

for i, row in test.iterrows():
    for pair in ldamodel.get_document_topics(test_bow[i]):
        col= 'Topic '+ str(pair[0])
        test.loc[i, col]= pair[1]

print('Topic distributions added to testing data')
"""

Question length created for training data
Question length created for testing data
Max IR score and corresponding page created for training data
Max IR score and corresponding page created for testing data
paren_match created for training data
Topics created for training data
Topics created for testing data


"\n# Topic distributions (training data)\nfor i, row in train.iterrows():\n    for pair in ldamodel[doc_term_matrix[i]]:\n        col= 'Topic '+ str(pair[0])\n        train.loc[i, col]= pair[1]\n        \nprint('Topic distributions added to training data')    \n    \n\n# Topic distributions (testing data)\n\nclean_test= [clean(row['Question Text']).split() for _, row in test.iterrows()]\ntest_bow= [dictionary.doc2bow(doc) for doc in clean_test]\n\nfor i, row in test.iterrows():\n    for pair in ldamodel.get_document_topics(test_bow[i]):\n        col= 'Topic '+ str(pair[0])\n        test.loc[i, col]= pair[1]\n\nprint('Topic distributions added to testing data')\n"

## Split into training and testing

In [10]:
X_train, X_test, y_train, y_test = train_test_split(train, train['paren_match'], test_size= 0.2, random_state= 4)
print('X train shape: {}'.format(X_train.shape))
print('y train shape: {}'.format(y_train.shape))
print('X test shape: {}'.format(X_test.shape))
print('y test shape: {}'.format(y_test.shape))

X_train= X_train.reset_index(drop= True)
y_train= y_train.reset_index(drop= True)
X_test= X_test.reset_index(drop= True)
y_test= y_test.reset_index(drop= True)

print('reindexed X train and y train for WOE embeddings')
print('reindexed X test and y test for WOE embeddings')

X train shape: (6528, 19)
y train shape: (6528,)
X test shape: (1632, 19)
y test shape: (1632,)
reindexed X train and y train for WOE embeddings
reindexed X test and y test for WOE embeddings


In [11]:
encoding= ce.WOEEncoder(cols= ['category_embed'])
encoding.fit(X_train[['category_embed']], X_train['paren_match'])
X_train['category_embed']=encoding.transform(X_train[['category_embed']])
print('Category embeddings created for training data')

# Create a new column to embed categories (testing data)
X_test['category_embed']= encoding.transform(X_test[['category_embed']])
print('Category embeddings created for testing data')

Category embeddings created for training data
Category embeddings created for testing data


In [14]:
import nltk
nltk.download('wordnet')

"""
Add new feature: topics (topic modeling)
We have 4 categories in the dataset. SO lets use 4 topics
"""
# Preprocessing the text first
clean_questions= [clean(row['Question Text']).split() for _, row in X_train.iterrows()]
print('Text preprocessing done')

# Create a word to index mapping
dictionary= corpora.Dictionary(clean_questions)
print('Word index dictionary created')

# Convert the list of documents to a BOW matrix using the dictionary prepared above
doc_term_matrix = [dictionary.doc2bow(doc) for doc in clean_questions]
print('BOW matrix created')

# Train the LDA model on the document word matrix
ldamodel = gensim.models.ldamodel.LdaModel(doc_term_matrix, num_topics=4, id2word = dictionary, passes=50)
print('LDA model trained')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
Text preprocessing done
Word index dictionary created
BOW matrix created
LDA model trained


In [15]:
import nltk
nltk.download('wordnet')

"""
Add topic distributions as features to train and test data
"""
# Topic distributions (training data)
for i, row in X_train.iterrows():
    for pair in ldamodel[doc_term_matrix[i]]:
        col= 'Topic '+ str(pair[0])
        X_train.loc[i, col]= pair[1]
        
print('Topic distributions added to training data')    
    

# Topic distributions (testing data)

clean_test= [clean(row['Question Text']).split() for _, row in X_test.iterrows()]
new_bow= [dictionary.doc2bow(doc) for doc in clean_test]

for i, row in X_test.iterrows():
    for pair in ldamodel.get_document_topics(new_bow[i]):
        col= 'Topic '+ str(pair[0])
        X_test.loc[i, col]= pair[1]

print('Topic distributions added to testing data')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Topic distributions added to training data
Topic distributions added to testing data


In [16]:
print(X_train.head(15))

    Question ID  ...   Topic 3
0        153256  ...  0.000000
1        109300  ...  0.000000
2        115761  ...  0.348513
3        184101  ...  0.011198
4        176257  ...  0.180916
5        101321  ...  0.000000
6          4402  ...  0.000000
7        118036  ...  0.028714
8        168293  ...  0.773437
9        105649  ...  0.000000
10        10999  ...  0.230344
11         7072  ...  0.000000
12       154257  ...  0.566846
13         4408  ...  0.012110
14       182705  ...  0.471777

[15 rows x 19 columns]


### features

In [0]:
features= ['Quest len', 'Page score',  'category_embed', 'Score difference', 'Topic 0', 'Topic 1', 'Topic 2', 'Topic 3']
target= ['paren_match']

In [0]:
#print(y_train)

### regularize data

In [0]:
"""
feat_reg= ['Quest len', 'Page score',  'category_embed', 'Score difference']

scaler= StandardScaler()

scaler.fit(X_train[feat_reg].values)

X_train[feat_reg]= scaler.transform(X_train[feat_reg].values)
X_test[feat_reg]= scaler.transform(X_test[feat_reg].values)

print('regularized X train and X test')
"""

y_train= np.reshape(y_train.values, (y_train.shape[0], ))

### SVM

In [20]:
svm_clf= SVC(kernel= 'rbf')

corr, incorr= auto_ML(svm_clf, features, X_train, y_train, X_test, y_test, folds= 10)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default 

Test prediction:
[0 0 0 ... 1 1 1]
-------------------------------------
Accuracy score: 0.8045343137254902
-------------------------------------
confusion matrix:
 [[ 255  235]
 [  84 1058]]
-------------------------------------
Cross Validation score: 0.7827818171715302


In [150]:
corr.describe()

,Page score,Score difference,category_embed,Topic 0,Topic 1,Topic 2,Topic 3
count,1312.000000,1312.000000,1312.000000,1312.000000,1312.000000,1312.000000,1312.000000
mean,37.294122,11.180870,0.039555,0.316499,0.161893,0.297133,0.216502
std,55.164444,20.035686,0.316887,0.348754,0.300512,0.364445,0.302805
min,0.000000,0.000000,-0.607354,0.000000,0.000000,0.000000,0.000000
25%,5.629610,1.509999,-0.308357,0.011156,0.000000,0.000000,0.000000
50%,8.803140,3.987098,0.139803,0.171268,0.014080,0.073319,0.031450
75%,45.515676,8.829453,0.355110,0.595032,0.117665,0.569518,0.371850
max,313.621279,175.442343,0.355110,0.990343,0.988549,0.991099,0.991211


In [151]:
incorr.describe()

,Page score,Score difference,category_embed,Topic 0,Topic 1,Topic 2,Topic 3
count,320.000000,320.000000,320.000000,320.000000,320.000000,320.000000,320.000000
mean,40.244534,3.882680,-0.110158,0.276087,0.268614,0.247621,0.200916
std,43.673396,6.592605,0.356306,0.351645,0.376792,0.344368,0.305104
min,-1.000000,0.000000,-0.607354,0.000000,0.000000,0.000000,0.000000
25%,4.807185,0.485593,-0.308357,0.012191,0.000000,0.000000,0.000000
50%,24.703295,1.273520,-0.308357,0.044640,0.026256,0.031940,0.020687
75%,64.112870,4.445684,0.139803,0.538449,0.442999,0.393972,0.350941
max,249.660259,55.106620,0.355110,0.988216,0.988894,0.987527,0.988449


In [152]:
incorr[incorr['category']== 'social']['Question Text']

1       the angelic liturgy is a minor text in this bo...
21      as a baby this hero was left to die on a mount...
45      one part of this work uses the metaphor of the...
52      she turned a gardener into a frog after accept...
64      according to one tradition it is the last of 1...
80      he was the human grandfather of melicertes who...
86      this deity owns hildesvini the battle boar whi...
104     robert kapsis argued that it was low in certai...
126     its name means house of stone although it is n...
134     he was the human grandfather of melicertes who...
169     cities on its shores include kibwesa wapembe a...
185     in robert paul wolff 's reconstruction and cri...
195     one son of this deity whacked travelers with a...
200     a pope named for this figures greek name had a...
222     eight of the ten books of the rig veda begin w...
227     among this hero 's appearances in homer is as ...
235     this figure was associated with the sycamore t...
248     one wo

In [153]:
X_test['category'].value_counts()

lit        611
history    461
social     360
science    200
Name: category, dtype: int64

In [154]:
X_train.head(15)

,Question ID,Question Text,QANTA Scores,Answer,Sentence Position,IR_Wiki Scores,category,Quest len,Wiki_page,Wiki_page_embed,Wiki_page2,Page score,Score difference,paren_match,category_embed,Topic 0,Topic 1,Topic 2,Topic 3
0,153256,the name originally belonged to a chief of the...,"aaron_burr:0.194713107831, tammany_hall:0.0927...",tammany_hall,1,"aaron_burr:5.77478343774, alexander_hamilton:4...",history,282,aaron_burr,aaron_burr,alexander_hamilton,5.774783,1.650241,0,0.355110,0.000000,0.000000,0.971204,0.000000
1,109300,its leader was killed by mengli-geray after a ...,"golden_horde:0.921987159189, kublai_khan:0.002...",golden_horde,2,"golden_horde:10.7242201667, genghis_khan:2.749...",history,409,golden_horde,golden_horde,genghis_khan,10.724220,7.974561,1,0.355110,0.000000,0.000000,0.981098,0.000000
2,115761,one resident of this state served as secretary...,"maine:0.0513319951667, william_mckinley:0.0512...",maine,3,"hubert_humphrey:3.50538015338, george_wallace:...",history,454,hubert_humphrey,hubert_humphrey,george_wallace,3.505380,0.533030,0,0.355110,0.000000,0.000000,0.982056,0.000000
3,184101,the wu-yang solution to the yang-mills field e...,"magnetic_monopole:0.806297691429, poynting_vec...",magnetic_monopole,1,"quantum_hall_effect:36.477413, displacement_cu...",science,240,magnetic_monopole,magnetic_monopole,photon,68.566459,13.538780,1,-0.607354,0.011311,0.966536,0.011082,0.011071
4,176257,wolfgang ketterle was awarded a nobel prize fo...,"bose-einstein_condensate:0.265955065478, satye...",bose-einstein_condensate,1,"james_clerk_maxwell:32.158349, albert_schweitz...",science,301,rubidium,rubidium,fermionic_condensate,68.990852,0.923727,0,-0.607354,0.176174,0.808232,0.000000,0.000000
5,101321,one derivative of this molecule is the mushroo...,"hydrazine:0.262803870649, azide:0.038854836003...",hydrazine,3,"glucose:66.824918, combustion:89.666787, aldeh...",science,695,hydrazine,hydrazine,bleach,190.988280,86.687014,1,-0.607354,0.000000,0.987076,0.000000,0.000000
6,4402,one important figure in this conflict later le...,"taiping_rebellion:0.0565281519044, tang_dynast...",taiping_rebellion,3,"taiping_rebellion:4.4651097851, boxer_rebellio...",history,319,taiping_rebellion,taiping_rebellion,boxer_rebellion,4.465110,1.228577,1,0.355110,0.000000,0.000000,0.977494,0.000000
7,118036,ostwald ripening is a consequence of this quan...,"ammonia:0.258013455934, heat_capacity:0.069410...",gibbs_free_energy,0,"magnetic_field:13.457197, rayleigh_scattering:...",science,82,momentum,momentum,quantum_dot,21.709716,2.180019,0,-0.607354,0.028448,0.914861,0.028157,0.028534
8,168293,the critic edward garnett encouraged this man ...,"joseph_conrad:0.165919060316, arthur_miller:0....",joseph_conrad,2,"joseph_conrad:9.04399973674, d._h._lawrence:2....",lit,372,joseph_conrad,joseph_conrad,d._h._lawrence,9.044000,6.227725,1,0.139803,0.978911,0.000000,0.000000,0.000000
9,105649,one version of it named for entner and doudoro...,"glycolysis:0.164142172849, rubisco:0.083784352...",glycolysis,1,"beta_oxidation:45.796817, dna_replication:50.2...",science,330,glycolysis,glycolysis,hexokinase,104.950963,47.707552,1,-0.607354,0.000000,0.974720,0.000000,0.000000


### Feedforward net

In [0]:
"""
Prepare input
"""
Xffn= np.array(X_train[features])
yffn= np.array(y_train)

Xtestffn= np.array(X_test[features])
ytestffn= np.array(y_test)


In [21]:
tokenizer = Tokenizer(nb_words=2000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')
tokenizer.fit_on_texts(X_train['Question Text'].values)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [22]:
tokenizer

In [0]:
Xtrain = tokenizer.texts_to_sequences(X_train['Question Text'].values)
Xtest = tokenizer.texts_to_sequences(X_test['Question Text'].values)

X = pad_sequences(Xtrain,maxlen = 100,truncating = 'post')
Y = pad_sequences(Xtest, maxlen = 100, truncating = 'post')

In [24]:
embed_dim = 128
lstm_out = 196
batch_size = 32

model = Sequential()
model.add(Embedding(2000, embed_dim,input_length = X.shape[1], dropout = 0.2))
model.add(LSTM(lstm_out, dropout_U = 0.2, dropout_W = 0.2))
model.add(Dense(1,activation='softmax'))
model.compile(loss = 'mean_squared_error', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

W0818 03:47:00.093438 140176621692800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  
W0818 03:47:00.134445 140176621692800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0818 03:47:00.141868 140176621692800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

/usr/local/lib/python3.6/dist-

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 128)          256000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 197       
Total params: 510,997
Trainable params: 510,997
Non-trainable params: 0
_________________________________________________________________
None


In [25]:

model.fit(X, y_train, batch_size = batch_size, nb_epoch = 3, verbose = 2)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  
W0818 03:47:05.495073 140176621692800 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0818 03:47:06.114328 140176621692800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/3
 - 42s - loss: 0.3197 - acc: 0.6803
Epoch 2/3
 - 33s - loss: 0.3197 - acc: 0.6803
Epoch 3/3
 - 33s - loss: 0.3197 - acc: 0.6803


In [26]:
print(model)

In [27]:
score,acc = model.evaluate(Y, y_test, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))


Score: 0.30
Validation Accuracy: 0.70


In [197]:
X.shape

(6528, 147)